<a href="https://colab.research.google.com/github/tripatheea/transform/blob/main/curriculum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy import interpolate
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras import regularizers
import tensorflow.keras.initializers
from sklearn.model_selection import train_test_split

In [3]:
min_log_depth = 0
max_log_depth = 2

depths = 10**np.linspace(min_log_depth, max_log_depth, 20000)


In [4]:
min(depths), max(depths)

(1.0, 100.0)

In [27]:
breaks = [10.0, 20.0, 30.0, 50.0, 80.0]

breaks_idx = [0] + [np.argmin(np.abs(depths - b)) for b in breaks] + [len(depths) - 1]

breaks_idx

[0, 9999, 13010, 14770, 16989, 19030, 19999]

In [28]:
data = np.load(f'/content/drive/MyDrive/transform/depth_varied_1_to_100_injs.npz')

path = '/content/drive/MyDrive/transform/depths_varied_1_to_100/'



In [29]:
all_x, all_y = data['x'], data['y']

In [30]:
all_train_metrics = {'loss': [], 'accuracy': [], 'p_det': []}
all_val_metrics = {'loss': [], 'accuracy': [], 'p_det': []}
all_test_p_dets = []
all_test_fpr_s = []
all_test_tpr_s = []

In [37]:
def get_dataset(br_idx_i):
    shuffle = True

    current_iter_x, current_iter_y = all_x[breaks_idx[br_idx_i]:breaks_idx[br_idx_i + 1]], all_y[breaks_idx[br_idx_i]:breaks_idx[br_idx_i + 1]]

    x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(current_iter_x, current_iter_y, test_size=0.4, random_state=42, shuffle=shuffle)
    x_test_s, x_val_s, y_test_s, y_val_s = train_test_split(x_test_s, y_test_s, test_size=0.5, random_state=26, shuffle=shuffle)

    print(x_train_s.shape, x_val_s.shape, x_test_s.shape)

    batch_size = 128

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_s, y_train_s)).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_s, y_test_s)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_val_s, y_val_s)).batch(batch_size)

    return train_dataset, test_dataset, val_dataset, x_test_s, y_test_s

# checkpoint_filepath = f


def plot_history(history, test_dataset, x_test_s, y_test_s, title):

    plt.figure(figsize=(18, 4))

    plt.subplot(141)

    plt.plot(np.arange(0, EPOCHS), history.history['loss'], label="Training Loss", marker="x", ms=3)
    plt.plot(np.arange(0, EPOCHS), history.history['val_loss'], label="Validation Loss", marker="x", ms=3)

    plt.legend()

    plt.subplot(142)

    metric = 'p_det'

    plt.plot(np.arange(0, EPOCHS), history.history[metric], label="Training pdet", marker="x", ms=3)
    plt.plot(np.arange(0, EPOCHS), history.history['val_' + metric], label="Validation pdet", marker="x", ms=3)

    plt.ylim(0, 1)

    plt.legend()

    plt.subplot(143)

    metric = 'accuracy'

    plt.plot(np.arange(0, EPOCHS), history.history[metric], label="Training pdet", marker="x", ms=3)
    plt.plot(np.arange(0, EPOCHS), history.history['val_' + metric], label="Validation pdet", marker="x", ms=3)

    plt.ylim(0, 1)

    plt.legend()


    plt.subplot(144)
    result = tf_model.evaluate(test_dataset)

    test_predicted_ys = tf_model.predict(x_test_s)
    fpr, tpr, thresholds = roc_curve(y_test_s, test_predicted_ys)

    p_det = interpolate.interp1d(fpr, tpr)(0.01)

    print(f"\n\ntest p_det is {p_det:.4f}.\n\n")

    all_test_tpr_s.append(tpr)
    all_test_fpr_s.append(fpr)
    all_test_p_dets.append(p_det)

    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], ls="dotted", lw=0.5)
    plt.grid(alpha=0.3, ls="dotted")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")

    plt.suptitle(title)

    plt.tight_layout()

    for m in ['loss', 'accuracy', 'p_det']:
        all_train_metrics[m].append(history.history[m])
        all_val_metrics[m].append(history.history['val_' + m])

In [38]:
def get_p_det_metric(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    return np.interp(0.01, fpr, tpr)
    
def p_det(y_true, y_pred): 
    return tf.numpy_function(get_p_det_metric, (y_true, y_pred), tf.double)

def get_model():
    act = 'relu'

    reg = tf.keras.regularizers.L2(
        l2=0.000000001)

    input = tf.keras.Input(shape=(all_x.shape[1], all_x.shape[2], all_x.shape[3],))
    x_layer = tf.keras.layers.Conv1D(64, (100), strides=16, kernel_regularizer=reg, activation=act, kernel_initializer='he_normal')(input)
    x_layer = tf.keras.layers.Conv2D(64, (2, 16), kernel_regularizer=reg, strides=(1, 4), activation=act, kernel_initializer='he_normal')(x_layer)
    x_layer = tf.keras.layers.Flatten()(x_layer)
    # x = tf.keras.layers.Dense(16, activation=act)(x)
    x_layer = tf.keras.layers.Dense(32, activation=act, kernel_regularizer=reg)(x_layer)
    x_layer = tf.keras.layers.Dense(16, activation=act, kernel_regularizer=reg)(x_layer)
    x_layer = tf.keras.layers.Dense(8, activation=act, kernel_regularizer=reg)(x_layer)
    x_layer = tf.keras.layers.Dense(4, activation=act, kernel_regularizer=reg)(x_layer)
    x_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x_layer)

    tf_model = tf.keras.Model(inputs=input, outputs=x_layer)

    return tf_model

tf_model = get_model()

In [39]:
tf_model.compile(
                optimizer='adam',
                 loss='binary_crossentropy',
              metrics=['accuracy', p_det],
                )

tf_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10, 382, 4)]      0         
                                                                 
 conv1d_2 (Conv1D)           (None, 10, 18, 64)        25664     
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 1, 64)          131136    
                                                                 
 flatten_2 (Flatten)         (None, 576)               0         
                                                                 
 dense_10 (Dense)            (None, 32)                18464     
                                                                 
 dense_11 (Dense)            (None, 16)                528       
                                                                 
 dense_12 (Dense)            (None, 8)                 136 

In [42]:
early_stoping_callback = tf.keras.callbacks.EarlyStopping(
                                monitor='val_p_det',
                                min_delta=0.005,
                                patience=100,
                                verbose=0,
                                mode='max',
                                baseline=0.85,
                                restore_best_weights=False,
                                start_from_epoch=50
                            )

In [40]:
EPOCHS = 2500

for br_idx_i in range(len(breaks_idx) - 1):

    min_depth, max_depth = depths[breaks_idx[br_idx_i]], depths[breaks_idx[br_idx_i + 1]]

    print(breaks_idx[br_idx_i], breaks_idx[br_idx_i + 1])
    print(f"\ndepth = {min_depth:.2f}, {max_depth:.2f}\n")


    train_dataset, test_dataset, val_dataset, x_test_s, y_test_s = get_dataset(br_idx_i)


    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                    filepath="/content/drive/MyDrive/transform/depths_varied_1_to_100/weights/weights.depth_" + f"{min_depth:.2f}_{max_depth:.2f}" + "_epoch_{epoch:02d}-val_p_det_{val_p_det:.2f}.hdf5",
                                    save_weights_only=True,
                                    monitor='val_p_det',
                                    mode='max',
                                    save_best_only=True,
                                    save_freq='epoch',
                                    verbose=False)

    

    history = tf_model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset, callbacks=[model_checkpoint_callback, early_stoping_callback])

    plot_history(history, test_dataset, x_test_s, y_test_s, title=f"Depth = {min_depth:.2f}, {max_depth:.2f}")


    model_path = path + '/saved_models'
    !mkdir -p model_path
    tf_model.save(model_path + f'/my_model_depths_{min_depth:.2f}_{max_depth:.2f}')

    # break

0 9999

depth = 1.00, 10.00

(5999, 10, 382, 4) (2000, 10, 382, 4) (2000, 10, 382, 4)
Epoch 1/2500
47/47 [==============================] - 12s 221ms/step - loss: 0.4929 - accuracy: 0.7510 - p_det: 0.6043 - val_loss: 0.7003 - val_accuracy: 0.8350 - val_p_det: 0.5174
Epoch 2/2500
47/47 [==============================] - 9s 183ms/step - loss: 0.3576 - accuracy: 0.9432 - p_det: 0.9107 - val_loss: 0.2889 - val_accuracy: 0.9945 - val_p_det: 0.9951
Epoch 3/2500
47/47 [==============================] - 10s 216ms/step - loss: 0.2535 - accuracy: 0.9845 - p_det: 0.9881 - val_loss: 0.1978 - val_accuracy: 0.9935 - val_p_det: 0.9941
Epoch 4/2500
47/47 [==============================] - 9s 198ms/step - loss: 0.1475 - accuracy: 0.9880 - p_det: 0.9928 - val_loss: 0.0971 - val_accuracy: 0.9940 - val_p_det: 0.9842
Epoch 5/2500
47/47 [==============================] - 9s 181ms/step - loss: 1.7313 - accuracy: 0.9600 - p_det: 0.9258 - val_loss: 0.2222 - val_accuracy: 0.9470 - val_p_det: 0.9121
Epoch 6/2500

KeyboardInterrupt: ignored

In [ ]:
# all_train_metrics

In [ ]:
# all_fpr_s